In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='d083a218-a227-46c1-b8da-bc1aeb4e0612', project_access_token='p-86a8b18f435d909c61de3fdc2206bce722713ed5')
pc = project.project_context


In [2]:
!pip install scikit-learn --upgrade
!pip install scipy --upgrade
!pip install imblearn --upgrade
!pip install xgboost --upgrade

Requirement already up-to-date: scikit-learn in /opt/conda/envs/Python36/lib/python3.6/site-packages (0.23.2)
Requirement already up-to-date: scipy in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.5.2)
Requirement already up-to-date: imblearn in /opt/conda/envs/Python36/lib/python3.6/site-packages (0.0)
Requirement already up-to-date: xgboost in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.2.0)


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_validate
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from scipy import stats
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Linear Regression

In [ ]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/654986294958/master/train_dataset_digitalhouse.csv
df_training_dataset = pd.read_csv(r'train_dataset_digitalhouse.csv')
df_training_dataset=pd.get_dummies(df_training_dataset, columns=['GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH'])

features=['EDAD', 
          'AVG_DH',
          'MINUTES_DH', 
          'EXPERIENCIA',
       'GENERO_FEMENINO', 'GENERO_MASCULINO', 
        'RESIDENCIA_ARGENTINA','RESIDENCIA_BRAZIL', 'RESIDENCIA_MEXICO', 
        'NV_ESTUDIO_POST_GRADUATE','NV_ESTUDIO_TERTIARY', 'NV_ESTUDIO_UNIVERSITARY',
       'ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL',
       'ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING',
       'ESTUDIO_PREV_MARKETING', 
         #'TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO','TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION'
         ]
target='DIAS_EMP'


X=df_training_dataset.copy()[features]
y=df_training_dataset.copy()[target]

#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    X.loc[X[i].isnull(),[i]]=X[i].mean()

#Detectando outliers
#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    Q1 = X[i].quantile(0.25)
#    Q3 = X[i].quantile(0.75)
#    IQR = Q3 - Q1
#    X.loc[(X[i] < (Q1 - 1.5 * IQR)),[i]]=Q1 - 1.5 * IQR
    #X.loc[(X[i] < (Q1 - 1.5 * IQR)),[i]]=X.loc[(X[i] < (Q1 - 1.5 * IQR)),[i]]/X[i].std()
#    X.loc[(X[i] > (Q3 + 1.5 * IQR)),[i]]=Q3 + 1.5 * IQR
    #X.loc[(X[i] < (Q1 - 1.5 * IQR)),[i]]=X.loc[(X[i] < (Q1 - 1.5 * IQR)),[i]]/X[i].std()
    
imp = IterativeImputer(max_iter=25)
imp.fit(X)
X=pd.DataFrame(imp.transform(X),columns=X.columns)

#from sklearn.impute import KNNImputer
#imp=KNNImputer(n_neighbors=15, weights="uniform")
#imp.fit(X)
#X=pd.DataFrame(imp.transform(X),columns=X.columns) 

#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    X.loc[X[i].isnull(),[i]]=X[i].median()

for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
    X[i]=(X[i]-X[i].mean())/X[i].std()

#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    X[i]=(X[i]-X[i].min(axis=0))/(X[i].max(axis=0)-X[i].min(axis=0))

In [ ]:
probar=['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA','GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH']
combinaciones=[]
for i in range(10):
    for j in combinations(probar,i):
        if len(list(j))>0:
            combinaciones.append(list(j))
            
genero=['GENERO_MASCULINO','GENERO_FEMENINO']
residencia=['RESIDENCIA_ARGENTINA','RESIDENCIA_BRAZIL','RESIDENCIA_MEXICO']
nv_estudio=['NV_ESTUDIO_POST_GRADUATE','NV_ESTUDIO_TERTIARY','NV_ESTUDIO_UNIVERSITARY']
estudio_prev=['ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL','ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING','ESTUDIO_PREV_MARKETING']
track_dh=['TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO','TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION']
            
probar=[]
for i in combinaciones:
    lista=i
    if 'GENERO' in i:
        lista.remove('GENERO')
        lista.extend(genero)
    if 'RESIDENCIA' in i:
        lista.remove('RESIDENCIA')
        lista.extend(residencia)
    if 'NV_ESTUDIO' in i:
        lista.remove('NV_ESTUDIO')
        lista.extend(nv_estudio)
    if 'ESTUDIO_PREV' in i:
        lista.remove('ESTUDIO_PREV')
        lista.extend(estudio_prev)
    if 'TRACK_DH' in i:
        lista.remove('TRACK_DH')
        lista.extend(track_dh)
    probar.append(lista)

In [ ]:
#promedios=[]
#for grupo in probar:
#    X=df_training_dataset[grupo]
#    y=df_training_dataset['DIAS_EMP']
#    poly = PolynomialFeatures(2)
#    X=poly.fit_transform(X)
#    lr= LinearRegression()
#    cv=cross_validate(lr,X,y,cv=10,scoring='r2')
#    promedios.append([grupo,cv['test_score'].mean()])

In [ ]:
#pd.DataFrame(promedios).sort_values(by=1,ascending=False).head().loc[486,:]

In [ ]:
poly = PolynomialFeatures(2)
X=poly.fit_transform(X)
lr= LinearRegression()
cv=cross_validate(lr,X,y,cv=10,scoring='r2')
print(cv['test_score'].mean())

# KNN

In [ ]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/654986294958/master/train_dataset_digitalhouse.csv
df_training_dataset = pd.read_csv(r'train_dataset_digitalhouse.csv')
df_training_dataset=pd.get_dummies(df_training_dataset, columns=['GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH'])

#imputar

for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
    df_training_dataset.loc[df_training_dataset[i].isnull(),[i]]=df_training_dataset[i].mean()
    
#from sklearn.impute import KNNImputer
#imp=KNNImputer(n_neighbors=30, weights="uniform")
#imp.fit(df_training_dataset)
#df_training_dataset=pd.DataFrame(imp.transform(df_training_dataset),columns=df_training_dataset.columns)    

#from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import IterativeImputer
#imp = IterativeImputer(max_iter=30, random_state=0)
#imp.fit(df_training_dataset)
#df_training_dataset=pd.DataFrame(imp.transform(df_training_dataset),columns=df_training_dataset.columns) 


#escalar
#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    df_training_dataset[i]=(df_training_dataset[i]-df_training_dataset[i].mean())/df_training_dataset[i].std()
    
#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    df_training_dataset[i]=(df_training_dataset[i]-df_training_dataset[i].min(axis=0))/(df_training_dataset[i].max(axis=0)-df_training_dataset[i].min(axis=0))


features=['EDAD', 
          #'AVG_DH',
          'MINUTES_DH', 'EXPERIENCIA',
       'GENERO_FEMENINO', 'GENERO_MASCULINO', 'RESIDENCIA_ARGENTINA',
       #'RESIDENCIA_BRAZIL', 'RESIDENCIA_MEXICO', 'NV_ESTUDIO_POST_GRADUATE',
       'NV_ESTUDIO_TERTIARY', 'NV_ESTUDIO_UNIVERSITARY',
       'ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL',
       'ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING',
       'ESTUDIO_PREV_MARKETING', 
        #'TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO','TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION'
         ]
target='DIAS_EMP'

In [ ]:
probar=['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA','GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH']
combinaciones=[]
for i in range(10):
    for j in combinations(probar,i):
        if len(list(j))>0:
            combinaciones.append(list(j))
            
genero=['GENERO_MASCULINO','GENERO_FEMENINO']
residencia=['RESIDENCIA_ARGENTINA','RESIDENCIA_BRAZIL','RESIDENCIA_MEXICO']
nv_estudio=['NV_ESTUDIO_POST_GRADUATE','NV_ESTUDIO_TERTIARY','NV_ESTUDIO_UNIVERSITARY']
estudio_prev=['ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL','ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING','ESTUDIO_PREV_MARKETING']
track_dh=['TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO','TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION']
            
probar=[]
for i in combinaciones:
    lista=i
    if 'GENERO' in i:
        lista.remove('GENERO')
        lista.extend(genero)
    if 'RESIDENCIA' in i:
        lista.remove('RESIDENCIA')
        lista.extend(residencia)
    if 'NV_ESTUDIO' in i:
        lista.remove('NV_ESTUDIO')
        lista.extend(nv_estudio)
    if 'ESTUDIO_PREV' in i:
        lista.remove('ESTUDIO_PREV')
        lista.extend(estudio_prev)
    if 'TRACK_DH' in i:
        lista.remove('TRACK_DH')
        lista.extend(track_dh)
    probar.append(lista)

In [ ]:
#promedios=[]
#hola=0
#for grupo in probar:
#    X=df_training_dataset[grupo]
#    y=df_training_dataset['DIAS_EMP']
#    knn= KNeighborsRegressor(n_neighbors=10)
#    cv=cross_validate(knn,X,y,cv=10,scoring='r2')
#    promedios.append([grupo,cv['test_score'].mean()])
#    print(hola)
#    hola=hola+1

In [ ]:
#pd.DataFrame(promedios).sort_values(by=1,ascending=False).head().loc[419,0]

In [ ]:
#promedios=[]
#for i in range(1,30):
#    X=df_training_dataset[features]
#    y=df_training_dataset[target]
#    knn= KNeighborsRegressor(n_neighbors=i)
#    cv=cross_validate(knn,X,y,cv=10,scoring='r2')
#    promedios.append([i,cv['test_score'].mean()])
#pd.DataFrame(promedios).sort_values(by=1,ascending=False)

In [ ]:
X=df_training_dataset[features]
y=df_training_dataset[target]
knn= KNeighborsRegressor(n_neighbors=19)
cv=cross_validate(knn,X,y,cv=10,scoring='r2')
print(cv['test_score'].mean())

# DecisionTree

In [ ]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/654986294958/master/train_dataset_digitalhouse.csv
df_training_dataset = pd.read_csv(r'train_dataset_digitalhouse.csv')
df_training_dataset=pd.get_dummies(df_training_dataset, columns=['GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH'])

#imputar

#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    df_training_dataset.loc[df_training_dataset[i].isnull(),[i]]=df_training_dataset[i].median()
    
#from sklearn.impute import KNNImputer
#imp=KNNImputer(n_neighbors=30, weights="uniform")
#imp.fit(df_training_dataset)
#df_training_dataset=pd.DataFrame(imp.transform(df_training_dataset),columns=df_training_dataset.columns)    

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=18, random_state=0)
imp.fit(df_training_dataset)
df_training_dataset=pd.DataFrame(imp.transform(df_training_dataset),columns=df_training_dataset.columns) 


#escalar

#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    df_training_dataset[i]=(df_training_dataset[i]-df_training_dataset[i].mean())/df_training_dataset[i].std()
    
#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    df_training_dataset[i]=(df_training_dataset[i]-df_training_dataset[i].min(axis=0))/(df_training_dataset[i].max(axis=0)-df_training_dataset[i].min(axis=0))


features=['EDAD', 
          #'AVG_DH',
          #'MINUTES_DH', 
          #'EXPERIENCIA',
       'GENERO_FEMENINO', 'GENERO_MASCULINO', 
    #'RESIDENCIA_ARGENTINA','RESIDENCIA_BRAZIL', 'RESIDENCIA_MEXICO', 'NV_ESTUDIO_POST_GRADUATE',
       'NV_ESTUDIO_TERTIARY', 'NV_ESTUDIO_UNIVERSITARY',
       'ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL',
       'ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING',
       'ESTUDIO_PREV_MARKETING', 
         #'TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO','TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION'
         ]
target='DIAS_EMP'

In [ ]:
probar=['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA','GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH']
combinaciones=[]
for i in range(10):
    for j in combinations(probar,i):
        if len(list(j))>0:
            combinaciones.append(list(j))
            
genero=['GENERO_MASCULINO','GENERO_FEMENINO']
residencia=['RESIDENCIA_ARGENTINA','RESIDENCIA_BRAZIL','RESIDENCIA_MEXICO']
nv_estudio=['NV_ESTUDIO_POST_GRADUATE','NV_ESTUDIO_TERTIARY','NV_ESTUDIO_UNIVERSITARY']
estudio_prev=['ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL','ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING','ESTUDIO_PREV_MARKETING']
track_dh=['TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO','TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION']
            
probar=[]
for i in combinaciones:
    lista=i
    if 'GENERO' in i:
        lista.remove('GENERO')
        lista.extend(genero)
    if 'RESIDENCIA' in i:
        lista.remove('RESIDENCIA')
        lista.extend(residencia)
    if 'NV_ESTUDIO' in i:
        lista.remove('NV_ESTUDIO')
        lista.extend(nv_estudio)
    if 'ESTUDIO_PREV' in i:
        lista.remove('ESTUDIO_PREV')
        lista.extend(estudio_prev)
    if 'TRACK_DH' in i:
        lista.remove('TRACK_DH')
        lista.extend(track_dh)
    probar.append(lista)

In [ ]:
promedios=[]
hola=0
for grupo in probar:
    X=df_training_dataset[grupo]
    y=df_training_dataset[target]
    dtr= DecisionTreeRegressor(max_depth=8)
    cv=cross_validate(dtr,X,y,cv=10,scoring='r2')
    promedios.append([grupo,cv['test_score'].mean()])
    #print(hola)
    #hola=hola+1

In [ ]:
pd.DataFrame(promedios).sort_values(by=1,ascending=False).loc[178,0]

In [ ]:
#X=df_training_dataset[features]
#y=df_training_dataset[target]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
#dtr= DecisionTreeRegressor()
#values={'max_depth':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28],'max_features':[1,2,3,4,5,6,7,8,9,10]}
#grid=GridSearchCV(dtr, param_grid = values,scoring = 'r2')
#grid.fit(X_train,y_train)
#print(grid.best_params_)

In [ ]:
X=df_training_dataset[features]
y=df_training_dataset[target]
dtr= DecisionTreeRegressor(max_depth=7,max_features=8)
cv=cross_validate(dtr,X,y,cv=10,scoring='r2')
print(cv['test_score'].mean())

# Lasso

In [ ]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/654986294958/master/train_dataset_digitalhouse.csv
df_training_dataset = pd.read_csv(r'train_dataset_digitalhouse.csv')
df_training_dataset=pd.get_dummies(df_training_dataset, columns=['GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH'])

#imputar

#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    df_training_dataset.loc[df_training_dataset[i].isnull(),[i]]=df_training_dataset[i].median()
    
#from sklearn.impute import KNNImputer
#imp=KNNImputer(n_neighbors=5, weights="uniform")
#imp.fit(df_training_dataset)
#df_training_dataset=pd.DataFrame(imp.transform(df_training_dataset),columns=df_training_dataset.columns)    

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=18, random_state=0)
imp.fit(df_training_dataset)
df_training_dataset=pd.DataFrame(imp.transform(df_training_dataset),columns=df_training_dataset.columns) 


#escalar

#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    df_training_dataset[i]=(df_training_dataset[i]-df_training_dataset[i].mean())/df_training_dataset[i].std()
    
#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    df_training_dataset[i]=(df_training_dataset[i]-df_training_dataset[i].min(axis=0))/(df_training_dataset[i].max(axis=0)-df_training_dataset[i].min(axis=0))


features=['EDAD', 
          'AVG_DH',
          'MINUTES_DH', 
          #'EXPERIENCIA',
       'GENERO_FEMENINO', 'GENERO_MASCULINO', 
    #'RESIDENCIA_ARGENTINA','RESIDENCIA_BRAZIL', 'RESIDENCIA_MEXICO', 
    'NV_ESTUDIO_POST_GRADUATE','NV_ESTUDIO_TERTIARY', 'NV_ESTUDIO_UNIVERSITARY',
       'ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL',
       'ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING',
       'ESTUDIO_PREV_MARKETING', 
         #'TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO','TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION'
         ]
target='DIAS_EMP'

In [ ]:
probar=['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA','GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH']
combinaciones=[]
for i in range(10):
    for j in combinations(probar,i):
        if len(list(j))>0:
            combinaciones.append(list(j))
            
genero=['GENERO_MASCULINO','GENERO_FEMENINO']
residencia=['RESIDENCIA_ARGENTINA','RESIDENCIA_BRAZIL','RESIDENCIA_MEXICO']
nv_estudio=['NV_ESTUDIO_POST_GRADUATE','NV_ESTUDIO_TERTIARY','NV_ESTUDIO_UNIVERSITARY']
estudio_prev=['ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL','ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING','ESTUDIO_PREV_MARKETING']
track_dh=['TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO','TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION']
            
probar=[]
for i in combinaciones:
    lista=i
    if 'GENERO' in i:
        lista.remove('GENERO')
        lista.extend(genero)
    if 'RESIDENCIA' in i:
        lista.remove('RESIDENCIA')
        lista.extend(residencia)
    if 'NV_ESTUDIO' in i:
        lista.remove('NV_ESTUDIO')
        lista.extend(nv_estudio)
    if 'ESTUDIO_PREV' in i:
        lista.remove('ESTUDIO_PREV')
        lista.extend(estudio_prev)
    if 'TRACK_DH' in i:
        lista.remove('TRACK_DH')
        lista.extend(track_dh)
    probar.append(lista)

In [ ]:
promedios=[]
hola=0
for grupo in probar:
    X=df_training_dataset[grupo]
    y=df_training_dataset[target]
    lasso= linear_model.Lasso(alpha=0.01)
    cv=cross_validate(lasso,X,y,cv=10,scoring='r2')
    promedios.append([grupo,cv['test_score'].mean()])
    #print(hola)
    #hola=hola+1

In [ ]:
pd.DataFrame(promedios).sort_values(by=1,ascending=False).loc[394,0]

In [ ]:
X=df_training_dataset[features]
y=df_training_dataset[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
lasso= linear_model.Lasso()
values={'alpha':[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.5,1.8,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,20,30,50,100,1000,2000,10000]}
grid=GridSearchCV(lasso, param_grid = values,scoring = 'r2')
grid.fit(X_train,y_train)
print(grid.best_params_)

In [ ]:
X=df_training_dataset[features]
y=df_training_dataset[target]
lasso= linear_model.Lasso(alpha=0.01)
cv=cross_validate(lasso,X,y,cv=10,scoring='r2')
print([cv['test_score'].mean()])

# Ridge

In [ ]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/654986294958/master/train_dataset_digitalhouse.csv
df_training_dataset = pd.read_csv(r'train_dataset_digitalhouse.csv')
df_training_dataset=pd.get_dummies(df_training_dataset, columns=['GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH'])

#imputar

#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    df_training_dataset.loc[df_training_dataset[i].isnull(),[i]]=df_training_dataset[i].median()
    
#from sklearn.impute import KNNImputer
#imp=KNNImputer(n_neighbors=5, weights="uniform")
#imp.fit(df_training_dataset)
#df_training_dataset=pd.DataFrame(imp.transform(df_training_dataset),columns=df_training_dataset.columns)    

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=15, random_state=0)
imp.fit(df_training_dataset)
df_training_dataset=pd.DataFrame(imp.transform(df_training_dataset),columns=df_training_dataset.columns) 


#escalar

#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    df_training_dataset[i]=(df_training_dataset[i]-df_training_dataset[i].mean())/df_training_dataset[i].std()
    
#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    df_training_dataset[i]=(df_training_dataset[i]-df_training_dataset[i].min(axis=0))/(df_training_dataset[i].max(axis=0)-df_training_dataset[i].min(axis=0))


features=['EDAD', 
          'AVG_DH',
          'MINUTES_DH', 
          'EXPERIENCIA',
       'GENERO_FEMENINO', 'GENERO_MASCULINO', 
    'RESIDENCIA_ARGENTINA','RESIDENCIA_BRAZIL', 'RESIDENCIA_MEXICO', 
    'NV_ESTUDIO_POST_GRADUATE','NV_ESTUDIO_TERTIARY', 'NV_ESTUDIO_UNIVERSITARY',
       'ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL',
       'ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING',
       'ESTUDIO_PREV_MARKETING', 
         #'TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO','TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION'
         ]
target='DIAS_EMP'

In [ ]:
probar=['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA','GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH']
combinaciones=[]
for i in range(10):
    for j in combinations(probar,i):
        if len(list(j))>0:
            combinaciones.append(list(j))
            
genero=['GENERO_MASCULINO','GENERO_FEMENINO']
residencia=['RESIDENCIA_ARGENTINA','RESIDENCIA_BRAZIL','RESIDENCIA_MEXICO']
nv_estudio=['NV_ESTUDIO_POST_GRADUATE','NV_ESTUDIO_TERTIARY','NV_ESTUDIO_UNIVERSITARY']
estudio_prev=['ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL','ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING','ESTUDIO_PREV_MARKETING']
track_dh=['TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO','TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION']
            
probar=[]
for i in combinaciones:
    lista=i
    if 'GENERO' in i:
        lista.remove('GENERO')
        lista.extend(genero)
    if 'RESIDENCIA' in i:
        lista.remove('RESIDENCIA')
        lista.extend(residencia)
    if 'NV_ESTUDIO' in i:
        lista.remove('NV_ESTUDIO')
        lista.extend(nv_estudio)
    if 'ESTUDIO_PREV' in i:
        lista.remove('ESTUDIO_PREV')
        lista.extend(estudio_prev)
    if 'TRACK_DH' in i:
        lista.remove('TRACK_DH')
        lista.extend(track_dh)
    probar.append(lista)

In [ ]:
promedios=[]
hola=0
for grupo in probar:
    X=df_training_dataset[grupo]
    y=df_training_dataset[target]
    ridge= linear_model.Ridge(alpha=0.7)
    cv=cross_validate(ridge,X,y,cv=10,scoring='r2')
    promedios.append([grupo,cv['test_score'].mean()])
    #print(hola)
    #hola=hola+1

In [ ]:
pd.DataFrame(promedios).sort_values(by=1,ascending=False)

In [ ]:
X=df_training_dataset[features]
y=df_training_dataset[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
ridge= linear_model.Ridge()
values={'alpha':[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.5,1.8,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,20,30,50,100,1000,2000,10000]}
grid=GridSearchCV(ridge, param_grid = values,scoring = 'r2')
grid.fit(X_train,y_train)
print(grid.best_params_)

In [ ]:
X=df_training_dataset[features]
y=df_training_dataset[target]
ridge= linear_model.Ridge(alpha=0.7)
cv=cross_validate(ridge,X,y,cv=10,scoring='r2')
print([cv['test_score'].mean()])

# XGBoost

In [ ]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/654986294958/master/train_dataset_digitalhouse.csv
df_training_dataset = pd.read_csv(r'train_dataset_digitalhouse.csv')
df_training_dataset=pd.get_dummies(df_training_dataset, columns=['GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH'])

In [ ]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/654986294958/master/train_dataset_digitalhouse.csv
df_training_dataset = pd.read_csv(r'train_dataset_digitalhouse.csv')
df_training_dataset=pd.get_dummies(df_training_dataset, columns=['GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH'])

#imputar

features=['EDAD', 
          'AVG_DH',
          'MINUTES_DH', 
          'EXPERIENCIA',
       'GENERO_FEMENINO', 'GENERO_MASCULINO', 
    'RESIDENCIA_ARGENTINA','RESIDENCIA_BRAZIL', 'RESIDENCIA_MEXICO', 
    'NV_ESTUDIO_POST_GRADUATE','NV_ESTUDIO_TERTIARY', 'NV_ESTUDIO_UNIVERSITARY',
       'ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL',
       'ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING',
       'ESTUDIO_PREV_MARKETING', 
         #'TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO','TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION'
         ]
target='DIAS_EMP'

X=df_training_dataset.copy()[features]
y=df_training_dataset.copy()[target]

#for i in ['EDAD',
#          #'AVG_DH','MINUTES_DH','EXPERIENCIA'
#         ]:
#    X.loc[X[i].isnull(),[i]]=X[i].mean()
    
#from sklearn.impute import KNNImputer
#imp=KNNImputer(n_neighbors=10, weights="uniform")
#imp.fit(df_training_dataset)
#df_training_dataset=pd.DataFrame(imp.transform(df_training_dataset),columns=df_training_dataset.columns) 

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=15, random_state=0)
imp.fit(X)
X=pd.DataFrame(imp.transform(X),columns=X.columns)

#iso = IsolationForest(contamination=0.3)
#yhat = iso.fit_predict(X)
#X.loc[yhat==-1,['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']]=np.nan


#from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import IterativeImputer
#imp = IterativeImputer(max_iter=15, random_state=0)
#imp.fit(X)
#X=pd.DataFrame(imp.transform(X),columns=X.columns)

#escalar

#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    df_training_dataset[i]=(df_training_dataset[i]-df_training_dataset[i].mean())/df_training_dataset[i].std()
    
#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    df_training_dataset[i]=(df_training_dataset[i]-df_training_dataset[i].min(axis=0))/(df_training_dataset[i].max(axis=0)-df_training_dataset[i].min(axis=0))


In [ ]:
df_training_dataset.loc[yhat==-1,['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']]=np.nan

In [ ]:
probar=['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA','GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH']
combinaciones=[]
for i in range(10):
    for j in combinations(probar,i):
        if len(list(j))>0:
            combinaciones.append(list(j))
            
genero=['GENERO_MASCULINO','GENERO_FEMENINO']
residencia=['RESIDENCIA_ARGENTINA','RESIDENCIA_BRAZIL','RESIDENCIA_MEXICO']
nv_estudio=['NV_ESTUDIO_POST_GRADUATE','NV_ESTUDIO_TERTIARY','NV_ESTUDIO_UNIVERSITARY']
estudio_prev=['ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL','ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING','ESTUDIO_PREV_MARKETING']
track_dh=['TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO','TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION']
            
probar=[]
for i in combinaciones:
    lista=i
    if 'GENERO' in i:
        lista.remove('GENERO')
        lista.extend(genero)
    if 'RESIDENCIA' in i:
        lista.remove('RESIDENCIA')
        lista.extend(residencia)
    if 'NV_ESTUDIO' in i:
        lista.remove('NV_ESTUDIO')
        lista.extend(nv_estudio)
    if 'ESTUDIO_PREV' in i:
        lista.remove('ESTUDIO_PREV')
        lista.extend(estudio_prev)
    if 'TRACK_DH' in i:
        lista.remove('TRACK_DH')
        lista.extend(track_dh)
    probar.append(lista)

In [ ]:
promedios=[]
hola=0
for grupo in probar:
    X=df_training_dataset[grupo]
    y=df_training_dataset[target]
    xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.09,max_depth = 5, n_estimators = 120,n_jobs=-1)
    cv=cross_validate(xgb_reg,X,y,cv=3,scoring='r2')
    promedios.append([grupo,cv['test_score'].mean()])
    print(hola)
    hola=hola+1

In [ ]:
pd.DataFrame(promedios).sort_values(by=1,ascending=False).loc[504,0]

In [ ]:
X=df_training_dataset[features]
y=df_training_dataset[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
xgb_reg = xgb.XGBRegressor(n_jobs=-1,learning_rate=0.09,max_depth=5)
values={'learning_rate':[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
       'max_depth':[5,6,7,8,9,10,11,12,13,14,15]}
grid=GridSearchCV(xgb_reg, param_grid = values,scoring = 'r2')
grid.fit(X_train,y_train)
print(grid.best_params_)

In [ ]:
#X=df_training_dataset[features]
#y=df_training_dataset[target]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.12,max_depth = 5, n_estimators = 120,n_jobs=-1)
cv=cross_validate(xgb_reg,X,y,cv=10,scoring='r2')
print([cv['test_score'].mean()])
print(cv['test_score'])

# Gradient Boosting

In [ ]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/654986294958/master/train_dataset_digitalhouse.csv
df_training_dataset = pd.read_csv(r'train_dataset_digitalhouse.csv')
df_training_dataset=pd.get_dummies(df_training_dataset, columns=['GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH'])

features=['EDAD', 
          'AVG_DH',
          'MINUTES_DH', 
          #'EXPERIENCIA',
        'GENERO_FEMENINO', 'GENERO_MASCULINO', 
        'RESIDENCIA_ARGENTINA','RESIDENCIA_BRAZIL', 'RESIDENCIA_MEXICO', 
        'NV_ESTUDIO_POST_GRADUATE','NV_ESTUDIO_TERTIARY', 'NV_ESTUDIO_UNIVERSITARY',
       'ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL',
       'ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING',
       'ESTUDIO_PREV_MARKETING', 
         #'TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO','TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION'
         ]
target='DIAS_EMP'


X=df_training_dataset.copy()[features]
y=df_training_dataset.copy()[target]

#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    X.loc[X[i].isnull(),[i]]=X[i].mean()
    
#from sklearn.impute import KNNImputer
#imp=KNNImputer(n_neighbors=10, weights="uniform")
#imp.fit(df_training_dataset)
#df_training_dataset=pd.DataFrame(imp.transform(df_training_dataset),columns=df_training_dataset.columns) 

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=15, random_state=0)
imp.fit(X)
X=pd.DataFrame(imp.transform(X),columns=X.columns)

#iso = IsolationForest(contamination=0.3)
#yhat = iso.fit_predict(X)
#X.loc[yhat==-1,['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']]=np.nan


#from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import IterativeImputer
#imp = IterativeImputer(max_iter=15, random_state=0)
#imp.fit(X)
#X=pd.DataFrame(imp.transform(X),columns=X.columns)

#escalar

#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    df_training_dataset[i]=(df_training_dataset[i]-df_training_dataset[i].mean())/df_training_dataset[i].std()
    
#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    df_training_dataset[i]=(df_training_dataset[i]-df_training_dataset[i].min(axis=0))/(df_training_dataset[i].max(axis=0)-df_training_dataset[i].min(axis=0))


In [ ]:
probar=['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA','GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH']
combinaciones=[]
for i in range(10):
    for j in combinations(probar,i):
        if len(list(j))>0:
            combinaciones.append(list(j))
            
genero=['GENERO_MASCULINO','GENERO_FEMENINO']
residencia=['RESIDENCIA_ARGENTINA','RESIDENCIA_BRAZIL','RESIDENCIA_MEXICO']
nv_estudio=['NV_ESTUDIO_POST_GRADUATE','NV_ESTUDIO_TERTIARY','NV_ESTUDIO_UNIVERSITARY']
estudio_prev=['ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL','ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING','ESTUDIO_PREV_MARKETING']
track_dh=['TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO','TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION']
            
probar=[]
for i in combinaciones:
    lista=i
    if 'GENERO' in i:
        lista.remove('GENERO')
        lista.extend(genero)
    if 'RESIDENCIA' in i:
        lista.remove('RESIDENCIA')
        lista.extend(residencia)
    if 'NV_ESTUDIO' in i:
        lista.remove('NV_ESTUDIO')
        lista.extend(nv_estudio)
    if 'ESTUDIO_PREV' in i:
        lista.remove('ESTUDIO_PREV')
        lista.extend(estudio_prev)
    if 'TRACK_DH' in i:
        lista.remove('TRACK_DH')
        lista.extend(track_dh)
    probar.append(lista)

In [ ]:
promedios=[]
hola=0
for grupo in probar:
    X=df_training_dataset[grupo]
    y=df_training_dataset[target]
    xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.09,max_depth = 5, n_estimators = 120,n_jobs=-1)
    cv=cross_validate(xgb_reg,X,y,cv=3,scoring='r2')
    promedios.append([grupo,cv['test_score'].mean()])
    print(hola)
    hola=hola+1

In [ ]:
pd.DataFrame(promedios).sort_values(by=1,ascending=False)

In [ ]:
X=df_training_dataset[features]
y=df_training_dataset[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
xgb_reg = xgb.XGBRegressor(n_jobs=-1,learning_rate=0.09,max_depth=5)
values={'learning_rate':[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
       'max_depth':[5,6,7,8,9,10,11,12,13,14,15]}
grid=GridSearchCV(xgb_reg, param_grid = values,scoring = 'r2')
grid.fit(X_train,y_train)
print(grid.best_params_)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
gbreg = GradientBoostingRegressor(learning_rate=0.13,n_estimators=90,max_depth=3,alpha=0.9)
cv=cross_validate(gbreg,X,y,cv=5,scoring='r2')
print([cv['test_score'].mean()])

# SVC

In [ ]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/654986294958/master/train_dataset_digitalhouse.csv
df_training_dataset = pd.read_csv(r'train_dataset_digitalhouse.csv')
df_training_dataset=pd.get_dummies(df_training_dataset, columns=['GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH'])

#imputar

features=['EDAD', 
          'AVG_DH',
          'MINUTES_DH', 
          'EXPERIENCIA',
       'GENERO_FEMENINO', 'GENERO_MASCULINO', 
    'RESIDENCIA_ARGENTINA','RESIDENCIA_BRAZIL', 'RESIDENCIA_MEXICO', 
    'NV_ESTUDIO_POST_GRADUATE','NV_ESTUDIO_TERTIARY', 'NV_ESTUDIO_UNIVERSITARY',
       'ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL',
       'ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING',
       'ESTUDIO_PREV_MARKETING', 
         #'TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO','TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION'
         ]
target='DIAS_EMP'

X=df_training_dataset.copy()[features]
y=df_training_dataset.copy()[target]

for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
    X.loc[X[i].isnull(),[i]]=X[i].mean()
    
#from sklearn.impute import KNNImputer
#imp=KNNImputer(n_neighbors=10, weights="uniform")
#imp.fit(df_training_dataset)
#df_training_dataset=pd.DataFrame(imp.transform(df_training_dataset),columns=df_training_dataset.columns) 

#from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import IterativeImputer
#imp = IterativeImputer(max_iter=15, random_state=0)
#imp.fit(X)
#X=pd.DataFrame(imp.transform(X),columns=X.columns)

#iso = IsolationForest(contamination=0.3)
#yhat = iso.fit_predict(X)
#X.loc[yhat==-1,['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']]=np.nan


#from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import IterativeImputer
#imp = IterativeImputer(max_iter=15, random_state=0)
#imp.fit(X)
#X=pd.DataFrame(imp.transform(X),columns=X.columns)

#escalar

for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
    df_training_dataset[i]=(df_training_dataset[i]-df_training_dataset[i].mean())/df_training_dataset[i].std()
    
#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    df_training_dataset[i]=(df_training_dataset[i]-df_training_dataset[i].min(axis=0))/(df_training_dataset[i].max(axis=0)-df_training_dataset[i].min(axis=0))

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
regr = SVR(kernel='rbf')
cv=cross_validate(regr,X,y,cv=5,scoring='r2')
print([cv['test_score'].mean()])

# USANDO MIS ULTIMAS ARMAS

In [4]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/654986294958/master/train_dataset_digitalhouse.csv
df_training_dataset = pd.read_csv(r'train_dataset_digitalhouse.csv')
df_training_dataset=pd.get_dummies(df_training_dataset, columns=['GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH'])

features=['EDAD', 
          'AVG_DH',
          'MINUTES_DH', 
          'EXPERIENCIA',
           'GENERO_FEMENINO', 
          'GENERO_MASCULINO', 
            'RESIDENCIA_ARGENTINA','RESIDENCIA_BRAZIL', 'RESIDENCIA_MEXICO', 
            'NV_ESTUDIO_POST_GRADUATE','NV_ESTUDIO_TERTIARY', 'NV_ESTUDIO_UNIVERSITARY',
            'ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL',
           'ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING',
           'ESTUDIO_PREV_MARKETING', 
             #'TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO','TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION'
         ]
target='DIAS_EMP'


X=df_training_dataset.copy()[features]
y=df_training_dataset.copy()[target]


Q1_edad = X['EDAD'].quantile(0.25)
Q3_edad = X['EDAD'].quantile(0.75)
IQR_edad = Q3_edad - Q1_edad
X['EDADINF']=np.where(X['EDAD'] < (Q1_edad - 1.5 * IQR_edad),1,0)
X['EDADSUP']=np.where(X['EDAD'] > (Q3_edad + 1.5 * IQR_edad),1,0)
X.loc[(X['EDAD'] < (Q1_edad - 1.5 * IQR_edad)),['EDAD']]=(Q1_edad - 1.5 * IQR_edad)
X.loc[(X['EDAD'] > (Q3_edad + 1.5 * IQR_edad)),['EDAD']]=(Q3_edad + 1.5 * IQR_edad)

Q1_avg = X['AVG_DH'].quantile(0.25)
Q3_avg = X['AVG_DH'].quantile(0.75)
IQR_avg = Q3_avg - Q1_avg
X['AVGINF']=np.where(X['AVG_DH'] < (Q1_avg - 1.5 * IQR_avg),1,0)
X['AVGSUP']=np.where(X['AVG_DH'] > (Q3_avg + 1.5 * IQR_avg),1,0)
X.loc[(X['AVG_DH'] < (Q1_avg - 1.5 * IQR_avg)),['AVG_DH']]=(Q1_avg - 1.5 * IQR_avg)
X.loc[(X['AVG_DH'] > (Q3_avg + 1.5 * IQR_avg)),['AVG_DH']]=(Q3_avg + 1.5 * IQR_avg)

Q1_min = X['MINUTES_DH'].quantile(0.25)
Q3_min = X['MINUTES_DH'].quantile(0.75)
IQR_min = Q3_min - Q1_min
#X['MININF']=np.where(X['MINUTES_DH'] < (Q1_avg - 1.5 * IQR_avg),1,0)
#X['MINSUP']=np.where(X['MINUTES_DH'] > (Q3_avg + 1.5 * IQR_avg),1,0)
X.loc[(X['MINUTES_DH'] < (Q1_min - 1.5 * IQR_min)),['MINUTES_DH']]=(Q1_min - 1.5 * IQR_min)
X.loc[(X['MINUTES_DH'] > (Q3_min + 1.5 * IQR_min)),['MINUTES_DH']]=(Q3_min + 1.5 * IQR_min)


Q1_exp = X['EXPERIENCIA'].quantile(0.25)
Q3_exp = X['EXPERIENCIA'].quantile(0.75)
IQR_exp = Q3_exp - Q1_exp
X.loc[(X['EXPERIENCIA'] < (Q1_exp - 1.5 * IQR_exp)),['EXPERIENCIA']]=(Q1_exp - 1.5 * IQR_exp)
X.loc[(X['EXPERIENCIA'] > (Q3_exp + 1.5 * IQR_exp)),['EXPERIENCIA']]=(Q3_exp + 1.5 * IQR_exp)

# #Detectando outliers
# for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#     Q1 = X[i].quantile(0.25)
#     Q3 = X[i].quantile(0.75)
#     IQR = Q3 - Q1
#     X.loc[(X[i] < (Q1 - 1.5 * IQR)),[i]]=(Q1 - 1.5 * IQR)
#     #X.loc[(X[i] < (Q1 - 1.5 * IQR)),[i]]=0
#     X.loc[(X[i] > (Q3 + 1.5* IQR)),[i]]=(Q3 + 1.5 * IQR)
#     #X.loc[(X[i] > (Q3 + 1.5 * IQR)),[i]]=0

X_imputar=X.copy()[['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA','GENERO_MASCULINO','GENERO_FEMENINO']]
imp = IterativeImputer(max_iter=15)
imp.fit(X_imputar)
X_imputar=pd.DataFrame(imp.transform(X_imputar),columns=X_imputar.columns)

for i in list(X_imputar.columns):
     X[i]=X_imputar[i]
    
#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    X[i]=(X[i]-X[i].mean())/X[i].std()

#for i in ['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA']:
#    X[i]=(X[i]-X[i].min(axis=0))/(X[i].max(axis=0)-X[i].min(axis=0))

#X.drop(labels=['ESTUDIO_PREV_MARKETING'],axis=1,inplace=True)

gbreg = xgb.XGBRegressor(learning_rate=0.14,n_estimators=90,max_depth=3,alpha=0.9,n_jobs=-1)
cv=cross_validate(gbreg,X,y,cv=15,scoring='r2')
print([cv['test_score'].mean()])
print([cv['test_score']])

--2020-09-18 06:36:45--  https://raw.githubusercontent.com/vanderlei-test/654986294958/master/train_dataset_digitalhouse.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 654532 (639K) [text/plain]
Saving to: ‘train_dataset_digitalhouse.csv.239’

100%[======================================>] 654,532     --.-K/s   in 0.02s   

2020-09-18 06:36:45 (26.4 MB/s) - ‘train_dataset_digitalhouse.csv.239’ saved [654532/654532]

[0.8100517042912898]
[array([0.80802775, 0.81373226, 0.8404568 , 0.81872969, 0.8040137 ,
       0.81576149, 0.80366462, 0.81095735, 0.80197893, 0.7865353 ,
       0.82868156, 0.79031392, 0.81263414, 0.81941666, 0.79587139])]


In [5]:
X.shape

(8995, 21)

In [6]:
gbreg_final=xgb.XGBRegressor(learning_rate=0.14,n_estimators=90,max_depth=3,alpha=0.9,n_jobs=-1)
gbreg_final.fit(X,y)

XGBRegressor(alpha=0.9, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.14, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=90, n_jobs=-1, num_parallel_tree=1, random_state=0,
             reg_alpha=0.899999976, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [ ]:
X.MINUTES_DH

<hr>

## Completar los datos necesarios para entregar la solución

### Como entrega de su solución, esperamos los resultados numéricos predichos por su modelo. Como entrada utilizará el archivo "to_be_scored.csv"

In [7]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/654986294958/master/to_be_scored_digitalhouse.csv
df_to_be_scored = pd.read_csv(r'to_be_scored_digitalhouse.csv')
df_to_be_scored.tail()

--2020-09-18 06:36:56--  https://raw.githubusercontent.com/vanderlei-test/654986294958/master/to_be_scored_digitalhouse.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67308 (66K) [text/plain]
Saving to: ‘to_be_scored_digitalhouse.csv.10’

100%[======================================>] 67,308      --.-K/s   in 0.004s  

2020-09-18 06:36:56 (18.2 MB/s) - ‘to_be_scored_digitalhouse.csv.10’ saved [67308/67308]



,Unnamed: 0,EDAD,GENERO,RESIDENCIA,NV_ESTUDIO,ESTUDIO_PREV,TRACK_DH,AVG_DH,MINUTES_DH,EXPERIENCIA,Unnamed: 10
995,995,33.0,MASCULINO,ARGENTINA,UNIVERSITARY,COMMERCIAL,PROGRAMACION,3.6,4576.5,15.1,NaN
996,996,40.0,MASCULINO,ARGENTINA,TERTIARY,COMMERCIAL,PROGRAMACION,3.4,4542.9,26.1,NaN
997,997,NaN,FEMENINO,ARGENTINA,UNIVERSITARY,ENGINEERING,DATA,3.4,NaN,27.4,NaN
998,998,NaN,MASCULINO,MEXICO,UNIVERSITARY,ENGINEERING,DATA,3.7,4730.4,1.1,NaN
999,999,36.0,NaN,ARGENTINA,UNIVERSITARY,NaN,PROGRAMACION,3.3,4506.5,20.4,NaN


In [8]:
df_to_be_scored.isnull().sum()

Unnamed: 0         0
EDAD             139
GENERO           132
RESIDENCIA       169
NV_ESTUDIO       142
ESTUDIO_PREV     140
TRACK_DH         153
AVG_DH           160
MINUTES_DH       164
EXPERIENCIA      145
Unnamed: 10     1000
dtype: int64

# ¡Atención!

### El marco de datos ``to_be_scored`` es su "hoja de evaluación". Tenga en cuenta que la columna "target" no existe en esta muestra, por lo que no se puede utilizar para modelos de entrenamiento basados en el aprendizaje supervisado.

# ¡Atención!

### Debes realizar los mismos pasos de procesamiento previo que hiciste en el conjunto de datos de entrenamiento antes de calificar la "hoja de respuestas"

In [9]:
df_to_be_scored=pd.get_dummies(df_to_be_scored, columns=['GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH'])

In [10]:
features_scored=['EDAD', 
          'AVG_DH',
          'MINUTES_DH', 
          'EXPERIENCIA',
           'GENERO_FEMENINO', 
          'GENERO_MASCULINO', 
            'RESIDENCIA_ARGENTINA','RESIDENCIA_BRAZIL', 'RESIDENCIA_MEXICO', 
            'NV_ESTUDIO_POST_GRADUATE','NV_ESTUDIO_TERTIARY', 'NV_ESTUDIO_UNIVERSITARY',
            'ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL',
           'ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING',
           'ESTUDIO_PREV_MARKETING']
X_scored=df_to_be_scored.copy()[features_scored]

X_scored['EDADINF']=np.where(X_scored['EDAD'] < (Q1_edad - 1.5 * IQR_edad),1,0)
X_scored['EDADSUP']=np.where(X_scored['EDAD'] > (Q3_edad + 1.5 * IQR_edad),1,0)
X_scored.loc[(X_scored['EDAD'] < (Q1_edad - 1.5 * IQR_edad)),['EDAD']]=(Q1_edad - 1.5 * IQR_edad)
X_scored.loc[(X_scored['EDAD'] > (Q3_edad + 1.5 * IQR_edad)),['EDAD']]=(Q3_edad + 1.5 * IQR_edad)

X_scored['AVGINF']=np.where(X_scored['AVG_DH'] < (Q1_avg - 1.5 * IQR_avg),1,0)
X_scored['AVGSUP']=np.where(X_scored['AVG_DH'] > (Q3_avg + 1.5 * IQR_avg),1,0)
X_scored.loc[(X_scored['AVG_DH'] < (Q1_avg - 1.5 * IQR_avg)),['AVG_DH']]=(Q1_avg - 1.5 * IQR_avg)
X_scored.loc[(X_scored['AVG_DH'] > (Q3_avg + 1.5 * IQR_avg)),['AVG_DH']]=(Q3_avg + 1.5 * IQR_avg)

X_scored.loc[(X_scored['MINUTES_DH'] < (Q1_min - 1.5 * IQR_min)),['MINUTES_DH']]=(Q1_min - 1.5 * IQR_min)
X_scored.loc[(X_scored['MINUTES_DH'] > (Q3_min + 1.5 * IQR_min)),['MINUTES_DH']]=(Q3_min + 1.5 * IQR_min)

X_scored.loc[(X_scored['EXPERIENCIA'] < (Q1_exp - 1.5 * IQR_exp)),['EXPERIENCIA']]=(Q1_exp - 1.5 * IQR_exp)
X_scored.loc[(X_scored['EXPERIENCIA'] > (Q3_exp + 1.5 * IQR_exp)),['EXPERIENCIA']]=(Q3_exp + 1.5 * IQR_exp)

In [11]:
X_imputar_scored=X_scored.copy()[['EDAD','AVG_DH','MINUTES_DH','EXPERIENCIA','GENERO_MASCULINO','GENERO_FEMENINO']]
X_imputar_scored=pd.DataFrame(imp.transform(X_imputar_scored),columns=X_imputar_scored.columns)
for i in list(X_imputar_scored.columns):
    X_scored[i]=X_imputar_scored[i]

<hr>

### Hacer las predicciones con el método "predict()" de sklearn y agregar los resultados en el marco de datos de la "hoja de evaluación"

In [12]:
y_pred = gbreg_final.predict(X_scored)
X_scored['target'] = y_pred
X_scored.tail()

,EDAD,AVG_DH,MINUTES_DH,EXPERIENCIA,GENERO_FEMENINO,GENERO_MASCULINO,RESIDENCIA_ARGENTINA,RESIDENCIA_BRAZIL,RESIDENCIA_MEXICO,NV_ESTUDIO_POST_GRADUATE,...,ESTUDIO_PREV_BUSINESS,ESTUDIO_PREV_COMMERCIAL,ESTUDIO_PREV_DEVELOPMENT,ESTUDIO_PREV_ENGINEERING,ESTUDIO_PREV_MARKETING,EDADINF,EDADSUP,AVGINF,AVGSUP,target
995,33.000000,3.6,4576.500000,15.1,0.0,1.0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,86.002945
996,40.000000,3.4,4542.900000,26.1,0.0,1.0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,93.251213
997,41.765904,3.4,4403.877191,27.4,1.0,0.0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,101.077599
998,23.620997,3.7,4730.400000,1.1,0.0,1.0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,83.133133
999,36.000000,3.3,4506.500000,20.4,0.0,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,91.182457


In [13]:
X_scored.shape

(1000, 22)

# ¡Atención!

### La columna agregada con los resultados debe llamarse "target", de lo contrario, su envío fallará.

<hr>

### Exportar el marco de datos de resultados como un archivo .csv a su proyecto de Watson Studio.

In [14]:
project.save_data(file_name="results.csv", data=X_scored.to_csv(index=False))

{'file_name': 'results.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ultimo-donotdelete-pr-zrfq3fvphc7tzf',
 'asset_id': 'c5e0e950-14b4-427c-a154-491209a9f345'}